<a href="https://colab.research.google.com/github/raketic-ognjen/Crypto-Portfolio-Optimization/blob/main/Crypto_Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font face="courier" size = 6 color = "Tangerine"><center>**Crypto Portfolio Optimization - Timespan 2020-2023**</center></font>

## <font face="courier" size = 4 color = "Tangerine"><center>**Core Idea**</center></font>

### I will test 3 crypto portfolios:

* 1st will be diversified:
    * Bitcoin (BTC)
    * Ethereum (ETH)
    * Binance Coin (BNB)
    * Cardano (ADA)
    * Solana (SOL)
    * Polkadot (DOT)
    * Chainlink (LINK)
    * Litecoin (LTC)
    * Ripple (XRP)
    * Stellar (XLM)

* 2nd will be focused on High Risk, High Reward:
   * Dogecoin (DOGE)
   * Shiba Inu (SHIB)
   * SafeMoon (SAFEMOON)
   * Polygon (MATIC)
   * VeChain (VET)
   * Theta (THETA)
   * Filecoin (FIL)
   * Aave (AAVE)
   * Decentraland (MANA)
   * SushiSwap (SUSHI)

* 3rd will be focused on NFT and DeFi:
    * Uniswap (UNI)
    * Compound (COMP)
    * Chainlink (LINK)
    * Aave (AAVE)
    * Maker (MKR)
    * Decentraland (MANA)
    * Enjin Coin (ENJ)
    * Flow (FLOW)
    * Axie Infinity (AXS)
    * The Sandbox (SAND)

I will try to test this 3 portfolio to see their frontiers and CAPM Models.

In [1]:
#Python Libraries
import yfinance as yf
import numpy as np
import numpy.linalg as LA
import scipy as sc
import scipy.optimize as sco
import scipy.stats as stat
import pandas as pd
import pandas_datareader as pdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn as skl
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import keras as k
import sympy as sp
import statsmodels.api as sm
import seaborn as sns

### Function Expected Return

In [2]:
def expRet(ret, n=252, com=False):
  """Calculates expected returns in one of two methods. Parameters are
    - ret: Pandas' DataFrame or Series of returns
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded return"""
  if com:
    return (1+ret).prod()**(n/len(ret))-1
  else:
    return ret.mean()*n

### Expected Return using Principal Component Function

In [3]:
def erPCA(ret,rf,f=252,n=None):
  if n!=None and ret.shape[1]<n:
    return print('Wrong input. Desired number of component n has to be smaller than or equal to number of stocks')
  else:
    l,evec=np.linalg.eig(ret.cov()*f)
    a=(evec[:,:np.count_nonzero(l>l.mean())] if n==None else evec[:,:n])
    rets_pca = ret@a
    errf,erpca=rf.mean(),rets_pca.mean()
    return[f*(errf+sm.OLS(ret[c]-rf,rets_pca).fit().params@erpca) for c in ret.columns]

### Annualized volatility function

In [4]:
def annualize_vol(ret,n=252):
  """Calculates volatility of sample returns. Parameters are:
    - ret: Pandas' DataFrame or Series of returns
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)"""
  return  ret.std()*(n**0.5)

### Expected Returns on Portfolio function

In [5]:
def per(W,ret=None, n=252,com=False,er_assumed=None):
  """Calculates expected returns of portfolio:
    - ret: data set of returns
    - W: iterable of weights
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - er_assumed: optional, assummed expected returns on each stock"""
  er = expRet(ret,n,com) if np.all(er_assumed==None) else er_assumed
  return er@W

### Variance on portfolio function

In [6]:
def pV(W,ret=None,n=252,vol=False,cov_assumed=None):
  """Calculates variance returns of portfoli:
    - ret: data set of returns
    - W: iterable of weights
    - n: optional, number of compounding periods in a year (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - vol: optional, determines whether you want to display portfolio's volatility (True) or variance (False)
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
  CovM=(ret.cov()*n if np.all(cov_assumed==None) else cov_assumed)
  return np.sqrt(W@CovM@W) if vol else W@CovM@W

### MVP function

In [7]:
def mvp(ret=None,f=252,com=False, cov_assumed=None):
  """Calculates MVP portfolio weights, volatility and expected returns for given data set of returns. Paramters are:
    - ret: optional, data set of returns. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded ret
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
  n=(len(ret.columns) if np.all(ret!=None) else len(cov_assumed))
  result=sco.minimize(lambda w: pV(w,ret,f,False,cov_assumed),[1/n]*n,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
  return dict(w=result.x,er=per(result.x,ret,f,com),vol=np.sqrt(result.fun)) if np.all(ret!=None) else result.x

### Optimal portfolio weights, volatility and expected returns Function

In [8]:
def targetP(ret=None, mi=None,Bounds=None,f=252,com=False,er_assumed=None,cov_assumed=None):
  """Calculates optimal portfolio weights, volatility and expected returns. Paramters are:
    - ret: optional, data set of returns. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - mi: target level of expected returns. If mi is omitted, function will caulculate MVP weights!
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - er_assumed: optional, assummed expected returns on each stock
    - cov_assumed: optional, assummed annualized covariance matrix of returns
    - Bounds: optional, simple list of length 2 (so that elements represent upper and lower bound) or complex list of length n
    (so that each sublist represent collection of upper and lower bound for each of n stocks separately) """
  n=(len(ret.columns) if np.all(ret!=None) else len(cov_assumed))
  if mi==None:
    result=sco.minimize(lambda w: pV(w,ret,f,False,cov_assumed),[1/n]*n,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
    if np.all(ret!=None):
      return dict(w=result.x, er=per(result.x,ret,f,com,er_assumed),vol=np.sqrt(result.fun))
    else:
      return dict(w=result.x,er=(None if np.all(er_assumed==None) else (result.x)@er_assumed),vol=np.sqrt(result.fun))
  elif type(mi)==float or type(mi)==np.float64:
    result= sco.minimize(lambda w: pV(w,ret,f,False,cov_assumed),[1/n]*n,
                         bounds=(None if Bounds== None else [Bounds]*n if len(Bounds)==2 else Bounds),
                    constraints=[dict(type='eq',fun=lambda w:sum(w)-1),
                                 dict(type='eq',fun=lambda w:per(w,ret,f,com,er_assumed)-mi)])
    return dict(w=result.x, er=mi, vol=np.sqrt(result.fun))
  else:
    return print('\33[91mWrong input given for parameter mi!\33[0m')

### Effective Frontier function

In [9]:
def EF(ret=None,Range=[0.01,0.4],plot=None,f=252,com=False,er_assumed=None,cov_assumed=None):
    """Prepares data for plotting efficient frontier or plot this curve (depending on what user chooses). Paramters are:
    - ret: optional, data set of returns. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - Range: optional, range in which you want to see efficient frontier.
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - plot: optional, determines whether you want to plot efficient frontier or not. Possible values are:
        a) None - on values will be displayed
        b) 'curve' - only curve will be plotted
        c) 'mvp' - curve with MVP will be plotted
        d) 'full' - curve with MVP will be plotted and efficient part will be emphasised
    - er_assumed: optional, assummed expected returns on each stock
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
    mi=np.arange(Range[0],Range[1]+0.01,0.01)
    sigma=np.array([targetP(ret,m,f=f,com=com,er_assumed=er_assumed,cov_assumed=cov_assumed)['vol'] for m in mi])
    if plot==None:
        return dict(sigma=sigma,er=mi)
    elif plot=='curve':
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3)))
        fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,max(0.25,np.max(sigma)+0.01)],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
        return fig
    elif plot=='mvp':
        M=targetP(ret,m,f=f,com=com,er_assumed=er_assumed,cov_assumed=cov_assumed)
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',width=3)))
        fig.add_trace(go.Scatter(x=[M['vol']],y=[M['er']],marker=dict(color='Red',line=dict(color='Black',width=2),size=10)))
        fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,max(0.25,np.max(sigma)+0.01)],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))
        return fig
    elif plot=='full':
        M=targetP(ret,f=f,com=com,er_assumed=er_assumed,cov_assumed=cov_assumed)
        fig=go.Figure()
        fig.add_trace(go.Scatter(x=sigma,y=mi,line=dict(color='Blue',dash='dash'),fill="tozeroy",fillcolor='Pink'))
        fig.add_trace(go.Scatter(x=sigma[mi>M['er']],y=mi[mi>M['er']],line=dict(color='Blue',width=3)))
        fig.add_trace(go.Scatter(x=[M['vol']],y=[M['er']],marker=dict(color='Red',line=dict(color='Black',width=2),size=10)))
        fig.update_layout(xaxis=dict(title_text='\$\sigma$',range=[-0.01,max(0.25,np.max(sigma)+0.01)],zerolinecolor='Black'),
                  yaxis=dict(title_text='$\mu$',zerolinecolor='Black'),showlegend=False,
                  title=dict(text="Efficient frontier",x=0.5,y=0.87,font=dict(size=25,color='Navy')))

        return fig
    else:
        return print("\33[91mWrong input ginven for parameter plot. Expected values are: None,'curve','mvp' or 'full'\33[0m")

### Max Sharpe Function

In [10]:
def maxSharpe(ret=None,rf=0.0,Bounds=[-1,1],f=252,com=False,er_assumed=None,cov_assumed=None):
    """Calculates weights for portfolio with maximal Sharpe's ratio, its volatility and expected returns. Paramters are:
    - ret: optional, data set of returns on risky assets. If it isn't provided, than it is required to provide er_assumed and cov_assumed.
    - f: optional, number of compounding periods in a year, i.e. frequency (252 for days, 52 for weeks, 12 for months, 4 for quarters...)
    - com: optional, deterimines whether expected returns are calculated as annualized sample mean or annualized compunded retu
    - rf: data set of returns on risk-free assets or assumed constant
    - er_assumed: optional, assummed expected returns on each stock
    - cov_assumed: optional, assummed annualized covariance matrix of returns"""
    n=(len(ret.columns) if np.all(ret!=None) else len(cov_assumed))
    r_f=(rf if type(rf)==float or type(rf)==np.float64 else expRet(rf))
    result=sco.minimize(lambda w: -(per(w,ret,f,com,er_assumed)-r_f)/pV(w,ret,f,True,cov_assumed),[1/n]*n,bounds=[Bounds]*n,
                        constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
    return dict(w=result.x, er=per(result.x,ret,f,com,er_assumed),vol=pV(result.x,ret,f,True,cov_assumed),sharpe=-result.fun)

### Portfolio tracking error Function

In [11]:
def portfolio_tracking_error(weights,ref_r,bb_r):
  return np.sqrt(((ref_r-(weights*bb_r).sum(axis=1))**2).sum())

### Style Analysis Function

In [12]:
def style_analysis(dependent_variable, explanatory_variables):
    """Returns the optimal weights that minimizes the Tracking error between a portfolio of the explanatory variables and the
    dependent variable. Parameters are:
    1. dependent_variable - data set of target returns
    2. explanatory_variables - data set of explanatory returns
    All parameters should be given as np.array, pd.Series or pd.DataFrame."""
    n=len(explanatory_variables.columns)
    result=sco.minimize(lambda w: portfolio_tracking_error(w,dependent_variable, explanatory_variables),
                        [1/n]*n,bounds=[(0,1)]*n,constraints=[dict(type='eq',fun=lambda w:sum(w)-1)])
    return pd.Series(result.x, index=explanatory_variables.columns)

### Cumulative simple returns Function

In [13]:
def compound(r):
    """returns the result of compounding the set of returns in r"""
    return np.expm1(np.log1p(r).sum())

### Implied Returns Function

In [14]:
def implied_returns(delta, sigma, w):
    """Obtain the implied expected returns by the market - i.e. construct vector Pi. Parameters are:
    - delta: Risk Aversion Coefficient (scalar)
    - sigma: Variance-Covariance Matrix (N x N) as pd.DataFrame
    - w: Portfolio weights (N x 1) as pd.Series
    Returns an N x 1 vector of Returns as pd.Series"""
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

### Proportional Prior function

In [15]:
def proportional_prior(sigma, tau, p):
    """Returns the He-Litterman simplified Omega. Parameters are:
    - sigma: N x N Covariance Matrix as pd.DataFrame
    - tau: a scalar
    - p: a K x N pd.DataFrame linking Q and Assets
    returns a P x P pd.DataFrame, a Matrix representing Prior Uncertainties"""
    helit_omega = p.dot(tau * sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index,columns=p.index)

### Black-Litterman model Function

In [16]:
def bl(w_prior, sigma_prior, p, q,omega=None,delta=2.5, tau=.02, rf=0):
    """Computes the posterior expected returns based on the original black litterman reference model. Parameters are:
    - W_prior: must be an N x 1 vector of weights, a pd.Series
    - Sigma_prior: is an N x N covariance matrix, a pd.DataFrame
    - P: must be a K x N matrix linking Q and the Assets, a pd.DataFrame
    - Q: must be an K x 1 vector of views, a pd.Series
    - Omega: must be a K x K matrix a pd.DataFrame, or None (in that case He-Litterman simplified Omega is used)
    - delta and tau: are scalars
    - rf: optional, assumed risk-free rate. By default it is set to 0 """
    if omega is None: omega = proportional_prior(sigma_prior, tau, p) #if omega isn't specified use He-Litterman siplification
    N = w_prior.shape[0] # How many assets do we have?
    K = q.shape[0] # And how many views?
    pi = implied_returns(delta, sigma_prior,  w_prior) # Determine market implied expected returns - Pi
    sigma_prior_scaled = tau * sigma_prior  # Adjust (scale) Sigma by the uncertainty scaling factor
    # estimating expected returns and covariance matrix by B-L model
    mu_bl=rf+pi+sigma_prior_scaled.dot(p.T).dot(LA.inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    sigma_bl=sigma_prior+sigma_prior_scaled-sigma_prior_scaled.dot(p.T).dot(LA.inv(p.dot(sigma_prior_scaled).dot(p.T)+omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

### Optimal (Tangent/Max Sharpe Ratio) Portfolio weights by using the Markowitz Optimization Procedure function

In [17]:
def w_msr(sigma, mu,rf=0):
    """Optimal (Tangent/Max Sharpe Ratio) Portfolio weights by using the Markowitz Optimization Procedure. Parameters are:
    - Mu: the vector of Excess expected returns given as pd.Series
    - Sigma: covariance matrix. Must be given as N x N matrix in pd.DataFrame"""
    invS=pd.DataFrame(LA.inv(sigma.values), index=sigma.columns, columns=sigma.index)
    w = invS.dot(mu-rf)
    return w/sum(w)

### W-Star function

In [18]:
def w_star(delta, sigma, mu):
    return (pd.DataFrame(LA.inv(sigma.values), index=sigma.columns, columns=sigma.index).dot(mu))/delta

# 1st portfolio

In [19]:
crypto1 = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'ADA-USD', 'SOL-USD', 'DOT-USD', 'LINK-USD', 'LTC-USD', 'XRP-USD', 'XLM-USD']

In [26]:
#Yf download
cryptoprice1 = yf.download(crypto1, start='2021-4-16')['Adj Close']
cryptoprice1.head()

[*********************100%%**********************]  10 of 10 completed


,ADA-USD,BNB-USD,BTC-USD,DOT-USD,ETH-USD,LINK-USD,LTC-USD,SOL-USD,XLM-USD,XRP-USD
Date,,,,,,,,,,
2021-04-16,1.416949,510.220184,61572.789062,41.537975,2431.946533,42.101555,310.612732,25.397366,0.613263,1.557527
2021-04-17,1.384869,520.416748,60683.820312,42.535950,2344.895020,40.372066,304.759918,25.099874,0.599255,1.560055
2021-04-18,1.275501,481.034241,56216.183594,37.347431,2237.136963,39.065910,273.098816,32.302917,0.544734,1.405998
2021-04-19,1.197959,505.092926,55724.265625,34.816669,2166.188721,35.879383,261.919220,31.675896,0.501234,1.315044
2021-04-20,1.265934,587.029114,56473.031250,35.444435,2330.210938,38.950634,260.941345,31.770054,0.527950,1.383299


In [27]:
rets1 = cryptoprice1.pct_change().dropna()
rets1

,ADA-USD,BNB-USD,BTC-USD,DOT-USD,ETH-USD,LINK-USD,LTC-USD,SOL-USD,XLM-USD,XRP-USD
Date,,,,,,,,,,
2021-04-17,-0.022640,0.019985,-0.014438,0.024026,-0.035795,-0.041079,-0.018843,-0.011713,-0.022842,0.001623
2021-04-18,-0.078974,-0.075675,-0.073622,-0.121980,-0.045954,-0.032353,-0.103889,0.286975,-0.090981,-0.098751
2021-04-19,-0.060793,0.050014,-0.008750,-0.067763,-0.031714,-0.081568,-0.040936,-0.019411,-0.079856,-0.064690
2021-04-20,0.056742,0.162220,0.013437,0.018031,0.075719,0.085599,-0.003733,0.002973,0.053300,0.051903
2021-04-21,-0.046763,-0.069853,-0.045454,-0.044391,0.014823,-0.064550,-0.008185,0.023386,-0.067927,-0.062749
...,...,...,...,...,...,...,...,...,...,...
2023-09-28,0.017684,0.014579,0.025380,0.021032,0.034674,0.023607,0.027596,0.043262,0.015176,0.019872
2023-09-29,0.001670,0.000946,-0.004064,0.000638,0.009112,0.016861,0.006028,0.017188,-0.011831,0.023932
2023-09-30,0.018078,-0.002128,0.002088,0.006955,0.001929,0.028465,0.006667,0.053914,-0.004511,-0.012102


In [60]:
#Weights for portfolio 1
w1 = [1/10]*10

#portfolio returns
crypto1rp = rets1@w1
crypto1rp

Date
2021-04-17   -0.012172
2021-04-18   -0.043520
2021-04-19   -0.040547
2021-04-20    0.051619
2021-04-21   -0.037166
                ...   
2023-09-28    0.024286
2023-09-29    0.006048
2023-09-30    0.009935
2023-10-01    0.034053
2023-10-02   -0.025994
Length: 899, dtype: float64

In [29]:
exr1 = expRet(rets1,252,True)
print(exr1)
print("*"*50)

anvol1 = annualize_vol(rets1,252)
print(anvol1)

ADA-USD    -0.377363
BNB-USD    -0.215520
BTC-USD    -0.199565
DOT-USD    -0.476410
ETH-USD    -0.101337
LINK-USD   -0.381832
LTC-USD    -0.352048
SOL-USD    -0.022566
XLM-USD    -0.379401
XRP-USD    -0.266728
dtype: float64
**************************************************
ADA-USD     0.771670
BNB-USD     0.658815
BTC-USD     0.521663
DOT-USD     0.842744
ETH-USD     0.684546
LINK-USD    0.868573
LTC-USD     0.764584
SOL-USD     1.023625
XLM-USD     0.778148
XRP-USD     0.865149
dtype: float64


In [30]:
#Portfolio expected return
crypto1per = per(W=w1,ret=rets1,com=True)
crypto1per

-0.27727702176125857

In [31]:
#pV
cryptopV1 = pV(w1,rets1,252,True)
cryptopV1

0.671755235844305

In [32]:
#MVP
crypto1mvp = mvp(rets1,252,True)
crypto1mvp

{'w': array([ 0.03103269,  0.29614544,  1.13485087, -0.14243568, -0.10982411,
        -0.12536536, -0.08943755, -0.04196004,  0.0963226 , -0.04932887]),
 'er': -0.16611155728392463,
 'vol': 0.4949266473909669}

In [33]:
#Manual optimization 1 - Only with pV
crypto1optres = sco.minimize(lambda w1: pV(w1,rets1),
                             [1/10]*10,
                             constraints=[dict(type='eq',fun=lambda w1: sum(w1)-1)])
crypto1optres

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.2449523862976625
       x: [ 3.103e-02  2.961e-01  1.135e+00 -1.424e-01 -1.098e-01
           -1.254e-01 -8.944e-02 -4.196e-02  9.632e-02 -4.933e-02]
     nit: 12
     jac: [ 4.898e-01  4.893e-01  4.901e-01  4.900e-01  4.901e-01
            4.897e-01  4.902e-01  4.899e-01  4.900e-01  4.897e-01]
    nfev: 133
    njev: 12

In [34]:
#Target of Portfolio Return
crypto1target = targetP(rets1,0.4)
crypto1target

{'w': array([-0.12494037,  0.39782742,  0.87965059, -0.61259607,  0.47419275,
        -0.15156823, -0.19950973,  0.26715226, -0.20533705,  0.27512842]),
 'er': 0.4,
 'vol': 0.6217833478891702}

In [35]:
#Manual optimization 2 - with pV and per
crypto1optres1 = sco.minimize(lambda w1: pV(w1,rets1),
                              [1/10]*10,
                              constraints=[dict(type='eq',fun=lambda w1:sum(w1)-1),
                                           dict(type='eq',fun=lambda w1:per(w1,rets1)-0.4)])
crypto1optres1

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.38661453171226495
       x: [-1.249e-01  3.978e-01  8.797e-01 -6.126e-01  4.742e-01
           -1.516e-01 -1.995e-01  2.672e-01 -2.053e-01  2.751e-01]
     nit: 12
     jac: [ 4.455e-01  5.341e-01  4.977e-01  3.849e-01  6.198e-01
            4.910e-01  4.692e-01  8.339e-01  4.419e-01  5.689e-01]
    nfev: 133
    njev: 12

In [36]:
#Manual optimization 3 -with pV and per and bounds
crypto1optres2 = sco.minimize(lambda w1: pV(w1,rets1),
                              [1/10]*10,
                              bounds=[[0,1.5]]*10,
                              constraints=[dict(type='eq',fun=lambda w1:sum(w1)-1),
                                           dict(type='eq',fun=lambda w1:per(w1,rets1)-0.4)])
crypto1optres2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7766688574462491
       x: [ 0.000e+00  0.000e+00  0.000e+00  1.129e-16  2.839e-01
            2.996e-16  0.000e+00  7.161e-01  2.332e-17  0.000e+00]
     nit: 3
     jac: [ 9.521e-01  8.337e-01  6.482e-01  1.094e+00  9.761e-01
            1.099e+00  9.383e-01  1.782e+00  8.568e-01  9.133e-01]
    nfev: 33
    njev: 3

In [37]:
#Effective Frontier without range
crypto1EF1 = EF(rets1,plot='full')
crypto1EF1

In [38]:
# With range
crypto1EF2 = EF(rets1,Range=[0.01,1.5],plot='full')
crypto1EF2

In [39]:
#Optimization for Sharpe's Ratio

crypto1opt = maxSharpe(rets1,0.02,com=True)
crypto1opt

{'w': array([-0.68323443,  1.        ,  1.        , -1.        ,  1.        ,
        -0.80743064, -0.51252131,  1.        , -0.99681362,  1.        ]),
 'er': 0.7954492922520281,
 'vol': 1.2467498409077502,
 'sharpe': 0.6219766522588274}

In [103]:
#Test for Efficient frontier

#initial Wealth
wealth1 = 100_000

##Determined amount invested in each stock
si1= crypto1opt['w']*wealth1

#Number of stocks
n1=si1/cryptoprice1.iloc[0]
n1

ADA-USD     -48218.701870
BNB-USD        195.993814
BTC-USD          1.624094
DOT-USD      -2407.435588
ETH-USD         41.119325
LINK-USD     -1917.816672
LTC-USD       -165.003317
SOL-USD       3937.416254
XLM-USD    -162542.596182
XRP-USD      64204.346674
Name: 2021-04-16 00:00:00, dtype: float64

In [104]:
#Value of portfolio each point in the time
ST1 = rets1@n1
ST1

Date
2021-04-17     4888.999881
2021-04-18    13742.045506
2021-04-19    12016.301385
2021-04-20    -8227.549236
2021-04-21     9578.030593
                  ...     
2023-09-28    -1969.318849
2023-09-29     3412.445187
2023-09-30     -775.992548
2023-10-01    -3740.441740
2023-10-02     3455.471454
Length: 899, dtype: float64

In [105]:
#preparing data
crypto1return1 = cryptoprice1.iloc[:,:10].pct_change()
crypto1return1.dropna(inplace=True)

#initial wealth
v1=100_000

#Optimal weights
crypto1mvp = mvp(crypto1return1)['w']
crypto1P40= targetP(crypto1return1,0.40)['w']

In [106]:
crypto1mvp

array([ 0.03103269,  0.29614544,  1.13485087, -0.14243568, -0.10982411,
       -0.12536536, -0.08943755, -0.04196004,  0.0963226 , -0.04932887])

In [107]:
crypto1P40

array([-0.12494037,  0.39782742,  0.87965059, -0.61259607,  0.47419275,
       -0.15156823, -0.19950973,  0.26715226, -0.20533705,  0.27512842])

In [109]:
#Vector of money invested into each asset
vcrypto1mvp1 = crypto1mvp*v1
vcrypto1P401 = crypto1P40*v1

In [110]:
vcrypto1mvp1

array([  3103.26945738,  29614.54437023, 113485.08747832, -14243.56834563,
       -10982.41050098, -12536.53596817,  -8943.75459738,  -4196.00415454,
         9632.25962514,  -4932.88736437])

In [111]:
vcrypto1P401

array([-12494.03701593,  39782.74217277,  87965.05943028, -61259.60675305,
        47419.27539315, -15156.82273416, -19950.97275406,  26715.22570398,
       -20533.70543883,  27512.84199584])

In [112]:
#Vector of innitial stock prices
crypto1p0= cryptoprice1.iloc[0,:10]
crypto1p0

ADA-USD         1.416949
BNB-USD       510.220184
BTC-USD     61572.789062
DOT-USD        41.537975
ETH-USD      2431.946533
LINK-USD       42.101555
LTC-USD       310.612732
SOL-USD        25.397366
XLM-USD         0.613263
XRP-USD         1.557527
Name: 2021-04-16 00:00:00, dtype: float64

In [98]:
#Number of shares
ncrypto1mvp1 = vcrypto1mvp1/crypto1p0
ncrypto1P401 = vcrypto1P401/crypto1p0

In [113]:
ncrypto1mvp1

ADA-USD      2190.106619
BNB-USD        58.042675
BTC-USD         1.843105
DOT-USD      -342.904733
ETH-USD        -4.515893
LINK-USD     -297.768954
LTC-USD       -28.793909
SOL-USD      -165.214150
XLM-USD     15706.571981
XRP-USD     -3167.128104
Name: 2021-04-16 00:00:00, dtype: float64

In [114]:
ncrypto1P401

ADA-USD     -8817.562748
BNB-USD        77.971714
BTC-USD         1.428635
DOT-USD     -1474.785574
ETH-USD        19.498486
LINK-USD     -360.006246
LTC-USD       -64.231021
SOL-USD      1051.889639
XLM-USD    -33482.706557
XRP-USD     17664.440455
Name: 2021-04-16 00:00:00, dtype: float64

In [115]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
Vcrypto1mvp1 = cryptoprice1.iloc[:,:10]@ncrypto1mvp1
Vcrypto1P401 = cryptoprice1.iloc[:,:10]@ncrypto1P401

In [116]:
figst = go.Figure()

figst.add_trace(
    go.Scatter(x=cryptoprice1.index,
               y=Vcrypto1mvp1,
               line=dict(color='Blue',width=3),
               name='MVP Portfolio')
)
figst.add_trace(
    go.Scatter(x=cryptoprice1.index,
               y=Vcrypto1P401,
               line=dict(color='Crimson',width=3),
               name='Target 40% pa')
)
figst.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 static strategies',
               x=0.5,
               y=0.87,
               font=dict(size=25,color='Navy'))
)

figst.show()

# 2nd portfolio

In [66]:
crypto2 = ['DOGE-USD', 'SHIB-USD', 'USDC-USD', 'MATIC-USD', 'VET-USD','THETA-USD','FIL-USD','AAVE-USD', 'MANA-USD','SUSHI-USD']
cryptoprice2 = yf.download(crypto2, start='2021-4-16')['Adj Close']
cryptoprice2.head()

[*********************100%%**********************]  10 of 10 completed


,AAVE-USD,DOGE-USD,FIL-USD,MANA-USD,MATIC-USD,SHIB-USD,SUSHI-USD,THETA-USD,USDC-USD,VET-USD
Date,,,,,,,,,,
2021-04-16,442.194397,0.365870,183.253845,1.313581,0.436447,1.000000e-06,16.315783,14.282880,1.001033,0.238089
2021-04-17,430.174683,0.284173,182.688660,1.526686,0.406253,2.000000e-06,16.020266,13.407032,1.010496,0.227585
2021-04-18,382.075104,0.320475,156.309326,1.349810,0.362901,2.000000e-06,14.037149,11.756942,1.000095,0.254632
2021-04-19,346.812561,0.407318,154.857834,1.317240,0.327200,4.000000e-06,12.199388,10.820083,0.999967,0.229339
2021-04-20,361.608185,0.319500,153.701645,1.396516,0.341524,2.000000e-06,12.706245,11.742767,1.000054,0.251997


In [67]:
rets2 = cryptoprice2.pct_change().dropna()
rets2

,AAVE-USD,DOGE-USD,FIL-USD,MANA-USD,MATIC-USD,SHIB-USD,SUSHI-USD,THETA-USD,USDC-USD,VET-USD
Date,,,,,,,,,,
2021-04-17,-0.027182,-0.223295,-0.003084,0.162232,-0.069181,1.000000,-0.018112,-0.061322,0.009453,-0.044118
2021-04-18,-0.111814,0.127746,-0.144395,-0.115856,-0.106712,0.000000,-0.123788,-0.123076,-0.010293,0.118843
2021-04-19,-0.092292,0.270982,-0.009286,-0.024129,-0.098377,1.000000,-0.130921,-0.079686,-0.000128,-0.099332
2021-04-20,0.042662,-0.215601,-0.007466,0.060183,0.043778,-0.500000,0.041548,0.085275,0.000087,0.098797
2021-04-21,-0.020657,-0.039358,-0.035288,-0.042022,0.051841,-0.500000,-0.016427,-0.057964,0.000080,-0.098279
...,...,...,...,...,...,...,...,...,...,...
2023-09-28,0.084424,0.016197,0.031027,0.019330,0.032087,0.000000,0.023312,0.021646,-0.000092,0.033586
2023-09-29,0.005542,0.011287,0.009242,0.056888,0.011873,0.000000,-0.003905,-0.000124,0.000031,0.007385
2023-09-30,0.021906,-0.000595,0.005203,-0.021623,0.014307,0.000000,0.017562,0.002010,0.000011,0.021406


In [68]:
#Returns on portfolio 2

#Weights for portfolio 2
w2 = [1/10]*10

#Portfolio returns
crypto2rp = rets2@w2
crypto2rp

Date
2021-04-17    0.072539
2021-04-18   -0.048934
2021-04-19    0.073683
2021-04-20   -0.035074
2021-04-21   -0.075807
                ...   
2023-09-28    0.026152
2023-09-29    0.009822
2023-09-30    0.006019
2023-10-01    0.044297
2023-10-02   -0.044060
Length: 899, dtype: float64

In [69]:
#Expected Returns
exr2 = expRet(rets2,252,True)
print(exr2)
print("*"*50)
anvol2 = annualize_vol(rets2,252)
print(anvol2)

AAVE-USD    -0.406620
DOGE-USD    -0.392991
FIL-USD     -0.674991
MANA-USD    -0.333370
MATIC-USD    0.065145
SHIB-USD     0.744017
SUSHI-USD   -0.606157
THETA-USD   -0.583478
USDC-USD    -0.000230
VET-USD     -0.524379
dtype: float64
**************************************************
AAVE-USD     0.946032
DOGE-USD     0.945148
FIL-USD      0.947630
MANA-USD     1.319179
MATIC-USD    1.116273
SHIB-USD     2.933713
SUSHI-USD    1.043319
THETA-USD    0.905994
USDC-USD     0.021122
VET-USD      0.858765
dtype: float64


In [70]:
#portfolio expected return
crypto2per = per(W=w2,ret=rets2,com=True)
crypto2per

-0.27130548655185033

In [71]:
#pV
cryptopV2 = pV(w2,rets2,252,True)
cryptopV2

0.762722156228521

In [72]:
#MVP
crypto2mvp = mvp(rets2,252,True)
crypto2mvp

{'w': array([-1.43418108e-03,  2.52039930e-03, -4.05331555e-03, -3.63670504e-04,
         3.67714911e-04, -2.35282362e-04,  2.30585892e-05, -1.18765199e-03,
         1.00012806e+00,  4.23487137e-03]),
 'er': 0.00052718012322364,
 'vol': 0.020831122337735847}

In [74]:
#Manual optimization 1 - Only with pV

crypto2optres = sco.minimize(lambda w2: pV(w2,rets2),
                             [1/10]*10,
                             constraints=[dict(type='eq',fun=lambda w2:sum(w2)-1)])
crypto2optres

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0004339356578497174
       x: [-1.434e-03  2.520e-03 -4.053e-03 -3.637e-04  3.677e-04
           -2.353e-04  2.306e-05 -1.188e-03  1.000e+00  4.235e-03]
     nit: 8
     jac: [ 9.564e-04  3.734e-04  5.071e-04  1.297e-03  7.101e-04
            7.392e-04  7.135e-04  8.426e-05  8.664e-04  1.125e-03]
    nfev: 91
    njev: 8

In [78]:
#Target of Portfolio Return
crypto2target = targetP(rets2,0.4)
crypto2target

{'w': array([ 0.03984587,  0.04150497, -0.12493026,  0.06250163,  0.1717858 ,
         0.03230376, -0.09356861, -0.09984137,  1.03728894, -0.06689073]),
 'er': 0.4,
 'vol': 0.19677703631445906}

In [82]:
#Manual optimization 2 - with pV and per
crypto2optres1 = sco.minimize(lambda w2: pV(w2,rets2),
                              [1/10]*10,
                              constraints=[dict(type='eq',fun=lambda w2:sum(w2)-1),
                                           dict(type='eq',fun=lambda w2:per(w2,rets2)-0.4)])
crypto2optres1.fun

0.038721202020701936

In [84]:
#Manual optimization 3 -with pV and per and bounds
crypto2optres2 = sco.minimize(lambda w2: pV(w2,rets2),
                              [1/10]*10,
                              bounds =[[0,1.5]]*10,
                              constraints=[dict(type='eq',fun=lambda w2:sum(w2)-1),
                                           dict(type='eq',fun=lambda w2: per(w2,rets2)-0.4)])
crypto2optres2

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.15535055332313458
       x: [ 0.000e+00  2.930e-16  0.000e+00  0.000e+00  1.536e-01
            1.116e-01  6.472e-17  0.000e+00  7.348e-01  3.570e-16]
     nit: 4
     jac: [ 3.430e-01  2.585e-01  3.010e-01  2.961e-01  4.999e-01
            2.086e+00  3.559e-01  3.019e-01  1.454e-03  2.895e-01]
    nfev: 44
    njev: 4

In [85]:
#Effective Frontier without range
crypto2EF1 = EF(rets2,plot='full')
crypto2EF1

In [87]:
# With range
crypto2EF2 = EF(rets2,Range=[0.01,1.5],plot='full')
crypto2EF2

In [88]:
#Optimization for Sharpe's Ratio
crypto2opt = maxSharpe(rets2,0.02,com=True)
crypto2opt

{'w': array([ 0.21997925,  0.22546477, -0.4986552 ,  0.21049731,  1.        ,
         0.13197398, -0.64677838, -0.39991902,  1.        , -0.24256271]),
 'er': 1.0040541572092454,
 'vol': 0.9565747323394465,
 'sharpe': 1.0287268981092506}

In [89]:
#Test for Efficient frontier

#initial Wealth
wealth2 = 100_000

##Determined amount invested in each stock
si2=crypto2opt['w']*wealth2

#Number of stocks
n2=si2/cryptoprice2.iloc[0]
n2

AAVE-USD     4.974718e+01
DOGE-USD     6.162429e+04
FIL-USD     -2.721117e+02
MANA-USD     1.602469e+04
MATIC-USD    2.291229e+05
SHIB-USD     1.319740e+10
SUSHI-USD   -3.964127e+03
THETA-USD   -2.799989e+03
USDC-USD     9.989681e+04
VET-USD     -1.018790e+05
Name: 2021-04-16 00:00:00, dtype: float64

In [90]:
#Value of portfolio each point in the time
ST2 = rets2@n2
ST2

Date
2021-04-17    1.319738e+10
2021-04-18   -3.070126e+04
2021-04-19    1.319740e+10
2021-04-20   -6.598712e+09
2021-04-21   -6.598680e+09
                  ...     
2023-09-28    5.071748e+03
2023-09-29    3.591813e+03
2023-09-30    6.395496e+02
2023-10-01    1.885359e+09
2023-10-02   -1.199076e+09
Length: 899, dtype: float64

In [91]:
#preparing data
crypto2return2 = cryptoprice2.iloc[:,:10].pct_change()
crypto2return2.dropna(inplace=True)

#initial wealth
v2 = 100_000

#Optimal weights

crypto2mvp = mvp(crypto2return2)['w']
crypto2P40 = targetP(crypto2return2,0.4)['w']

In [92]:
crypto2mvp

array([-1.43418108e-03,  2.52039930e-03, -4.05331555e-03, -3.63670504e-04,
        3.67714911e-04, -2.35282362e-04,  2.30585892e-05, -1.18765199e-03,
        1.00012806e+00,  4.23487137e-03])

In [93]:
crypto2P40

array([ 0.03984587,  0.04150497, -0.12493026,  0.06250163,  0.1717858 ,
        0.03230376, -0.09356861, -0.09984137,  1.03728894, -0.06689073])

In [94]:
#Vector of money invested into each asset
vcrypto2mvp = crypto2mvp*v2
vcrypto2P40 = crypto2P40*v2

In [95]:
vcrypto2mvp

array([-1.43418108e+02,  2.52039930e+02, -4.05331555e+02, -3.63670504e+01,
        3.67714911e+01, -2.35282362e+01,  2.30585892e+00, -1.18765199e+02,
        1.00012806e+05,  4.23487137e+02])

In [96]:
vcrypto2P40

array([  3984.5869187 ,   4150.496789  , -12493.02550243,   6250.16254341,
        17178.58035399,   3230.37638748,  -9356.86088043,  -9984.13747173,
       103728.89372421,  -6689.07286221])

In [97]:
#Vector of innitial stock prices
crypto2p0 = cryptoprice2.iloc[0,:10]
crypto2p0

AAVE-USD     4.421944e+02
DOGE-USD     3.658700e-01
FIL-USD      1.832538e+02
MANA-USD     1.313581e+00
MATIC-USD    4.364470e-01
SHIB-USD     1.000000e-06
SUSHI-USD    1.631578e+01
THETA-USD    1.428288e+01
USDC-USD     1.001033e+00
VET-USD      2.380890e-01
Name: 2021-04-16 00:00:00, dtype: float64

In [117]:
#Number of shares
ncrypto2mvp2 = vcrypto2mvp/crypto2p0
ncrypto2P40 = vcrypto2P40/crypto2p0

In [118]:
ncrypto2mvp2

AAVE-USD    -3.243327e-01
DOGE-USD     6.888784e+02
FIL-USD     -2.211858e+00
MANA-USD    -2.768543e+01
MATIC-USD    8.425191e+01
SHIB-USD    -2.352824e+07
SUSHI-USD    1.413269e-01
THETA-USD   -8.315214e+00
USDC-USD     9.990960e+04
VET-USD      1.778693e+03
Name: 2021-04-16 00:00:00, dtype: float64

In [119]:
ncrypto2P40

AAVE-USD     9.010939e+00
DOGE-USD     1.134418e+04
FIL-USD     -6.817333e+01
MANA-USD     4.758110e+03
MATIC-USD    3.936006e+04
SHIB-USD     3.230376e+09
SUSHI-USD   -5.734853e+02
THETA-USD   -6.990283e+02
USDC-USD     1.036219e+05
VET-USD     -2.809484e+04
Name: 2021-04-16 00:00:00, dtype: float64

In [120]:
#the number of shares that we need to hold over time, we can calculate the value of our portfolio on each date in testing sample simply by multiplying previous two
Vcrypto2mvp = cryptoprice2.iloc[:,:10]@ncrypto2mvp2
Vcrypto2P40 = cryptoprice2.iloc[:,:10]@ncrypto2P40

In [122]:
figst1 = go.Figure()

figst1.add_trace(
    go.Scatter(x=cryptoprice2.index,
               y=Vcrypto2mvp,
               line=dict(color='Blue',width=3),
               name='MVP Portfolio')
)
figst1.add_trace(
    go.Scatter(x=cryptoprice2.index,
               y=Vcrypto2P40,
               line=dict(color='Crimson',width=3),
               name='Target 40% pa')
)
figst1.update_layout(
    xaxis=dict(title_text='Date',zerolinecolor='Black'),
    yaxis=dict(title_text='Dollars',zerolinecolor='Black'),
    title=dict(text='Comparing 2 static strategies',
               x=0.5,
               y=0.87,
               font=dict(size=25,color='Navy'))
)
figst1.show()

# 3rd portfolio

In [58]:
crypto3 = ['UNI-USD', 'COMP', 'LINK-USD', 'AAVE-USD', 'MKR-USD', 'MANA-USD', 'ENJ-USD', 'FLOW-USD', 'AXS-USD', 'SAND']

cryptoprice3 = yf.download(crypto3, start='2021-4-16')['Adj Close']
cryptoprice3.head()

[*********************100%%**********************]  10 of 10 completed


,AAVE-USD,AXS-USD,COMP,ENJ-USD,FLOW-USD,LINK-USD,MANA-USD,MKR-USD,SAND,UNI-USD
Date,,,,,,,,,,
2021-04-16,442.194397,7.448085,18.030001,3.027652,36.581551,42.101555,1.313581,3381.328857,7.543564,0.001272
2021-04-17,430.174683,7.817194,NaN,2.999848,35.695862,40.372066,1.526686,3341.222900,NaN,0.001257
2021-04-18,382.075104,6.650603,NaN,2.694819,32.571568,39.065910,1.349810,3233.734863,NaN,0.000698
2021-04-19,346.812561,8.106076,17.780001,2.412772,32.903790,35.879383,1.317240,3520.074951,7.435520,0.001361
2021-04-20,361.608185,8.564796,17.219999,2.565713,34.380268,38.950634,1.396516,3452.383301,7.474809,0.001369


In [59]:
rets3 = cryptoprice3.pct_change().dropna()
rets3

,AAVE-USD,AXS-USD,COMP,ENJ-USD,FLOW-USD,LINK-USD,MANA-USD,MKR-USD,SAND,UNI-USD
Date,,,,,,,,,,
2021-04-17,-0.027182,0.049558,0.000000,-0.009183,-0.024211,-0.041079,0.162232,-0.011861,0.000000,-0.011792
2021-04-18,-0.111814,-0.149234,0.000000,-0.101681,-0.087525,-0.032353,-0.115856,-0.032170,0.000000,-0.444710
2021-04-19,-0.092292,0.218848,-0.013866,-0.104663,0.010200,-0.081568,-0.024129,0.088548,-0.014323,0.949857
2021-04-20,0.042662,0.056590,-0.031496,0.063388,0.044873,0.085599,0.060183,-0.019230,0.005284,0.005878
2021-04-21,-0.020657,-0.046243,0.027875,-0.032196,-0.027676,-0.064550,-0.042022,0.164467,0.034166,0.220599
...,...,...,...,...,...,...,...,...,...,...
2023-09-28,0.084424,0.016647,0.040956,0.006004,0.015275,0.023607,0.019330,0.011382,0.017204,0.025316
2023-09-29,0.005542,0.018550,-0.049180,0.003544,0.019065,0.016861,0.056888,-0.040891,-0.014799,0.000000
2023-09-30,0.021906,-0.002291,0.000000,0.006436,0.001044,0.028465,-0.021623,0.056322,0.000000,0.000000
